<a href="https://colab.research.google.com/github/Yamurlak1465/LearningAI/blob/main/ConvolutionalNeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Part 1**

In [53]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [54]:
# Convert MNIST image files into a Tensor of 4 dimensions (Number of images, Height, Width, Color Channels)
transform = transforms.ToTensor()

In [55]:
# Train Data
train_data = datasets.MNIST(root = "/cnn_data", train = True, download = True, transform = transform)

In [56]:
# Test Data
test_data = datasets.MNIST(root = "/cnn_data", train = False, download = True, transform = transform)

In [57]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [58]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

# **Part 2**

In [59]:
# Create a small bach size for images...let's say 10
train_loader = DataLoader(train_data, batch_size = 10, shuffle = True)
test_loader = DataLoader(test_data, batch_size = 10, shuffle = False)

In [60]:
# Define our CNN Model
# Describe Convolutional Layer and what it's doing (2 covolutional layers)
# This is just an example
conv1 = nn.Conv2d(1, 6, 3, 1)
conv2 = nn.Conv2d(6, 16, 3, 1)

In [61]:
# Get 1 MNIST image
for i, (X_Train, y_train) in enumerate(train_data):
  break

In [62]:
X_Train.shape

torch.Size([1, 28, 28])

In [63]:
x = X_Train.view(1, 1, 28, 28)

In [64]:
# Perform our first convolution
x = F.relu(conv1(x)) # Rectified Linear Unit for our activation function

In [65]:
# 1 single image,
# 6 is the filters we asked for,
# 26x26 are the pixels (It droped from 28x28 to 26x26 because we dont really need the outer frame becose there is no data there (Padding is off (We didn't set the Padding)))
x.shape

torch.Size([1, 6, 26, 26])

In [66]:
# Pass through the pooling layer
x = F.max_pool2d(x, 2, 2) # Kernel of 2 and Stride of 2

In [67]:
x.shape # 26 / 2 = 13

torch.Size([1, 6, 13, 13])

In [68]:
# Do our second convolutional layer
x = F.relu(conv2(x))

In [69]:
# We didn't set padding so we lose 2 pixels around the edges again
x.shape

torch.Size([1, 16, 11, 11])

In [70]:
# Second Pooling Layer
x = F.max_pool2d(x, 2, 2)

In [71]:
# 11 / 2 = 5.5 but we have to round down, because you can't invent data to round up
x.shape

torch.Size([1, 16, 5, 5])

In [72]:
((28 - 2) / 2 - 2) / 2

5.5

# **Part 3**

In [73]:
# Model Class
class ConvolutionalNeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 6, 3, 1)
    self.conv2 = nn.Conv2d(6, 16, 3, 1)

    # Fully Connected Layers
    self.fc1 = nn.Linear(5 * 5 * 16, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  # Forward Function
  def forward(self, X):
    X = F.relu(self.conv1(X))
    X = F.max_pool2d(X, 2, 2) # 2x2 Kernel and a Stride of 2
    # Secon Pass
    X = F.relu(self.conv2(X))
    X = F.max_pool2d(X, 2, 2) # 2x2 Kernel and a Stride of 2

    # Review to flatten it out
    X = X.view(-1, 116 * 5 * 5) # Negative one so that we can vary the batch size

    # Fully Connected Layers
    X = F.relu(self.fc1(X))
    X = F.relu(self.fc2(X))
    X = self.fc3(X)

    return F.log_softmax(X, dim = 1)

In [74]:
# Create an Instance of our Model
torch.manual_seed(41)
model = ConvolutionalNeuralNetwork()
model

ConvolutionalNeuralNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [75]:
# Loss Function Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001) # The smaller the Learning Rate, tle longer its gonna take to train.

# **Part 4**

In [76]:
import time
start_time = time.time()

# CREATE VARIABILES TO TRACK THINGS
epochs = 5
train_losses = []
test_losses = []
train_correct = []
test_correct = []



# CREATE A FOR LOOP OF EPOCHS
for i in range(epochs):
  trn_corr = 0
  tst_corr = 0



  # TRAIN
  for b,(X_train, y_train) in enumerate(train_loader):
    b+=1 # Start our batches at 1
    y_pred = model(X_train) # Get predicted values from the training set. Not flattened 2D
    loss = criterion(y_pred, y_train) # How off are we? We wanna compare the predictions to the correct answers in y_train

    predicted = torch.max(y_pred.date, 1)[1] # Add up the number of correct predictions. Indexed off the first point
    batch_correct = (predicted == y_train).sum() # How many we got correct from this specific batch. True = 1, False = 0, sum those up
    trn_corr += batch_correct # Keep track as we go along in training

    # Update our  parameters
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print out some results
    if b % 600 ==0:
      print(f'Epochs: {i}, Btach: {b}, Loss: {loss.item()}')

  train_losses.append(loss)
  train_correct.append(trn_corr)



  # TEST
  with torch.no_grad(): # No gradient so we don't update our weights and biases with test data
    for b,(X_test, y_test) in enumerate(test_loader):
      y_val = model(X_test)
      predicted = torch.max(y_val.data, 1)[1] # Adding up correct predictions
      tst_corr += (predicted == y_test).sum() # How many we got correct from this specific batch. True = 1, False = 0, sum those up

  loss = criterion(y_val, y_test)
  test_losses.append(loss)
  test_correct.append(tst_corr)


current_time = time.time()
total_time = current_time() - start_time()
print(f'Training Took: {total_time / 60} minutes!')


RuntimeError: shape '[-1, 100000]' is invalid for input of size 4000